In [ ]:
#delete the empty folder
import os
import shutil
data_dir = '/home/yuhaowang/data/processed_data'
for dir in os.listdir(data_dir):
    data_path = os.path.join(data_dir, dir,'output')
    for slide in os.listdir(data_path):
        slide_dir =os.path.join(data_path,slide)
        #print(slide_dir)
        #if slide_dir is not dir,remove it
        if not os.path.isdir(slide_dir):
            #print(slide_dir)
            os.remove(slide_dir)
            continue
        if len(os.listdir(slide_dir)) <3:
            print(slide_dir)
            #remove this empty dir
            #remove even it not empty
            shutil.rmtree(slide_dir)
            #break
        

In [1]:
import multiprocessing as mp
from pathlib import Path
import h5py
import os

def read_assets_from_h5(h5_path: str):
    """Read the assets from the h5 file"""
    assets = {}
    attrs = {}
    try:
        with h5py.File(h5_path, 'r') as f:
            for key in f.keys():
                assets[key] = f[key][:]
                if f[key].attrs is not None:
                    attrs[key] = dict(f[key].attrs)
        return assets, attrs
    except Exception as e:
        print(f'Error reading {h5_path}: {e}')
        return None, None

def process_file(file_path):
    try:
        data, _ = read_assets_from_h5(file_path)
        if data is None:
            print('Removing corrupt file:', file_path)
            os.remove(file_path)
            return

        feature = data.get('features')
        coords = data.get('coords')
        
        if coords is None or feature is None:
            print('Removing file with missing data:', file_path)
            os.remove(file_path)
            return

        if len(coords.shape) != 2:
            print('Removing file with invalid coords shape:', file_path)
            os.remove(file_path)
            return

        if feature.shape[0] <= 80:
            print('Removing file due to low dimension:', file_path)
            os.remove(file_path)
    except Exception as e:
        print(f'Error processing {file_path}: {e}')
        os.remove(file_path)

def main():
    h5_file_list = list(Path('/data4/embedding/temp').rglob('*.h5'))
    
    print(f'Total files found: {len(h5_file_list)}')
    
    with mp.Pool(processes=mp.cpu_count()) as pool:
        pool.map(process_file, h5_file_list)
    
if __name__ == "__main__":
    main()

Total files found: 39603
Removing file due to low dimension:Removing file due to low dimension:Removing file due to low dimension:   /data4/embedding/temp/embedding/private_chunk_4/UNI-2/S19071.h5/data4/embedding/temp/embedding/HIST2ST/UNI-2/C3.h5/data4/embedding/temp/embedding/private_chunk_4/UNI-2/S15659.h5


Removing file due to low dimension: /data4/embedding/temp/embedding/HIST2ST/UNI-2/A2.h5
Removing file due to low dimension: /data4/embedding/temp/embedding/private_chunk_3/UNI-2/S19812.h5
Removing file due to low dimension: /data4/embedding/temp/embedding/private_chunk_3/UNI-2/S06225.h5
Removing file due to low dimension: /data4/embedding/temp/embedding/private_chunk_4/UNI-2/S11574.h5
Removing file due to low dimension:Removing file due to low dimension: /data4/embedding/temp/embedding/private_chunk_3/UNI-2/S11585.h5
 /data4/embedding/temp/embedding/private_chunk_3/UNI-2/S11614.h5Removing file due to low dimension:
 /data4/embedding/temp/embedding/private_chunk_3/UNI-2/S09545.h5

# 下面这个板块要着重处理embedding命令的问题

In [1]:
#BCNB数据集 出来
import os
import re
#处理BCNB数据集
def rename_h5_files(root_folder):
    for foldername, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(".h5") and re.match(r"^\d", filename):  # 判断是否以数字开头
                old_path = os.path.join(foldername, filename)
                new_filename = f"bcnb_{filename}"
                new_path = os.path.join(foldername, new_filename)
                
                # 避免重复重命名
                if not os.path.exists(new_path):
                    os.rename(old_path, new_path)
                    print(f"Renamed: {old_path} -> {new_path}")
                else:
                    print(f"Skipping {old_path}, target file already exists.")

if __name__ == "__main__":
    root_directory = '/data4/embedding/BCNB'
    if os.path.exists(root_directory) and os.path.isdir(root_directory):
        rename_h5_files(root_directory)
    else:
        print("提供的路径无效，请输入正确的文件夹路径。")

Renamed: /data4/embedding/BCNB/FMBC/37.h5 -> /data4/embedding/BCNB/FMBC/bcnb_37.h5
Renamed: /data4/embedding/BCNB/FMBC/756.h5 -> /data4/embedding/BCNB/FMBC/bcnb_756.h5
Renamed: /data4/embedding/BCNB/FMBC/309.h5 -> /data4/embedding/BCNB/FMBC/bcnb_309.h5
Renamed: /data4/embedding/BCNB/FMBC/21.h5 -> /data4/embedding/BCNB/FMBC/bcnb_21.h5
Renamed: /data4/embedding/BCNB/FMBC/876.h5 -> /data4/embedding/BCNB/FMBC/bcnb_876.h5
Renamed: /data4/embedding/BCNB/FMBC/825.h5 -> /data4/embedding/BCNB/FMBC/bcnb_825.h5
Renamed: /data4/embedding/BCNB/FMBC/882.h5 -> /data4/embedding/BCNB/FMBC/bcnb_882.h5
Renamed: /data4/embedding/BCNB/FMBC/1050.h5 -> /data4/embedding/BCNB/FMBC/bcnb_1050.h5
Renamed: /data4/embedding/BCNB/FMBC/125.h5 -> /data4/embedding/BCNB/FMBC/bcnb_125.h5
Renamed: /data4/embedding/BCNB/FMBC/980.h5 -> /data4/embedding/BCNB/FMBC/bcnb_980.h5
Renamed: /data4/embedding/BCNB/FMBC/386.h5 -> /data4/embedding/BCNB/FMBC/bcnb_386.h5
Renamed: /data4/embedding/BCNB/FMBC/914.h5 -> /data4/embedding/BCNB

In [2]:
import os
import re

def rename_h5_files(root_folder):
    # 遍历所有子文件夹
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".h5"):
                old_path = os.path.join(subdir, file)
                
                # 使用正则表达式提取前缀部分
                match = re.match(r"(TCGA-[A-Z0-9]+-[A-Z0-9]+-[0-9]+)", file)
                if match:
                    new_name = match.group(1) + ".h5"  # 只保留前缀并加上 .h5
                    new_path = os.path.join(subdir, new_name)
                    
                    # 确保新文件名不重复
                    if old_path != new_path:
                        os.rename(old_path, new_path)
                        print(f"Renamed: {old_path} -> {new_path}")
                    else:
                        print(f"Skipped (already correct): {old_path}")

if __name__ == "__main__":
    root_folder = "/data4/embedding/TCGA-BRCA"  # 修改为你的文件夹路径
    rename_h5_files(root_folder)

Skipped (already correct): /data4/embedding/TCGA-BRCA/UNI-2/TCGA-C8-A27B-01.h5
Skipped (already correct): /data4/embedding/TCGA-BRCA/UNI-2/TCGA-BH-A0BF-01.h5
Skipped (already correct): /data4/embedding/TCGA-BRCA/UNI-2/TCGA-BH-A0C3-01.h5
Skipped (already correct): /data4/embedding/TCGA-BRCA/UNI-2/TCGA-A1-A0SI-01.h5
Skipped (already correct): /data4/embedding/TCGA-BRCA/UNI-2/TCGA-A7-A3J0-01.h5
Skipped (already correct): /data4/embedding/TCGA-BRCA/UNI-2/TCGA-AN-A0G0-01.h5
Skipped (already correct): /data4/embedding/TCGA-BRCA/UNI-2/TCGA-BH-A0B7-01.h5
Skipped (already correct): /data4/embedding/TCGA-BRCA/UNI-2/TCGA-AC-A2FM-01.h5
Skipped (already correct): /data4/embedding/TCGA-BRCA/UNI-2/TCGA-D8-A3Z5-01.h5
Skipped (already correct): /data4/embedding/TCGA-BRCA/UNI-2/TCGA-AR-A2LE-01.h5
Skipped (already correct): /data4/embedding/TCGA-BRCA/UNI-2/TCGA-A8-A08P-01.h5
Skipped (already correct): /data4/embedding/TCGA-BRCA/UNI-2/TCGA-E2-A14S-01.h5
Skipped (already correct): /data4/embedding/TCGA-BRC

In [3]:
#BCNB数据集 出来
import os
import re
#处理BCNB数据集
def rename_h5_files(root_folder):
    for foldername, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(".h5") and re.match(r"^\d", filename):  # 判断是否以数字开头
                old_path = os.path.join(foldername, filename)
                new_filename = f"AIDPATH_{filename}"
                new_path = os.path.join(foldername, new_filename)
                
                # 避免重复重命名
                if not os.path.exists(new_path):
                    os.rename(old_path, new_path)
                    print(f"Renamed: {old_path} -> {new_path}")
                else:
                    print(f"Skipping {old_path}, target file already exists.")

if __name__ == "__main__":
    root_directory = '/data4/embedding/AIDPATH'
    if os.path.exists(root_directory) and os.path.isdir(root_directory):
        rename_h5_files(root_directory)
    else:
        print("提供的路径无效，请输入正确的文件夹路径。")

Renamed: /data4/embedding/AIDPATH/FMBC/2058.h5 -> /data4/embedding/AIDPATH/FMBC/AIDPATH_2058.h5
Renamed: /data4/embedding/AIDPATH/FMBC/2034.h5 -> /data4/embedding/AIDPATH/FMBC/AIDPATH_2034.h5
Renamed: /data4/embedding/AIDPATH/FMBC/1916.h5 -> /data4/embedding/AIDPATH/FMBC/AIDPATH_1916.h5
Renamed: /data4/embedding/AIDPATH/FMBC/1991.h5 -> /data4/embedding/AIDPATH/FMBC/AIDPATH_1991.h5
Renamed: /data4/embedding/AIDPATH/FMBC/2063.h5 -> /data4/embedding/AIDPATH/FMBC/AIDPATH_2063.h5
Renamed: /data4/embedding/AIDPATH/FMBC/1818.h5 -> /data4/embedding/AIDPATH/FMBC/AIDPATH_1818.h5
Renamed: /data4/embedding/AIDPATH/FMBC/2051.h5 -> /data4/embedding/AIDPATH/FMBC/AIDPATH_2051.h5
Renamed: /data4/embedding/AIDPATH/FMBC/1882.h5 -> /data4/embedding/AIDPATH/FMBC/AIDPATH_1882.h5
Renamed: /data4/embedding/AIDPATH/FMBC/2068.h5 -> /data4/embedding/AIDPATH/FMBC/AIDPATH_2068.h5
Renamed: /data4/embedding/AIDPATH/FMBC/2074.h5 -> /data4/embedding/AIDPATH/FMBC/AIDPATH_2074.h5
Renamed: /data4/embedding/AIDPATH/FMBC/1

In [4]:
import h5py
def read_assets_from_h5( h5_path: str) -> tuple:
    '''Read the assets frdom the h5 file'''
    assets = {}
    attrs = {}
    with h5py.File(h5_path, 'r') as f:
        for key in f.keys():
            assets[key] = f[key][:]
            if f[key].attrs is not None:
                attrs[key] = dict(f[key].attrs)
    return assets, attrs

test_path = '/data4/embedding/AIDPATH/FMBC/1656.h5'
assets, attrs = read_assets_from_h5(test_path)
assets['features']

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/data4/embedding/AIDPATH/FMBC/1656.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## 删除某些测试的不太行的特征，重新提取


In [31]:
import os
import shutil
embedding_dir ='/data4/embedding'
for dataset in os.listdir(embedding_dir):
    dataset_path = os.path.join(embedding_dir, dataset)
    for model in os.listdir(dataset_path):
        if model == 'FMBC':
            print(os.path.join(dataset_path, model))
            shutil.rmtree(os.path.join(dataset_path, model))

/data4/embedding/TCGA-BRCA/FMBC
/data4/embedding/AIDPATH/FMBC
/data4/embedding/BRACS/FMBC
/data4/embedding/IMPRESS/FMBC
/data4/embedding/BCNB/FMBC
